In [91]:
from random import choices, randint, randrange, random, sample, seed 
from collections import namedtuple
from typing import List, Optional, Callable, Tuple
import numpy
from functools import partial
#Choices(, , # of draws from the population (2 , a pair)) is a function from the random module of python
# Use of tuples so that we can store /= types of data Vs array 
#Tuples are the fastest data structure 
#fix a seed for reproducitbilty later 
#===================================================#
import numpy as np # importing numpy for matrix operations 
from scipy import *
from scipy.optimize import minimize
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython import display 
import itertools
from IPython.display import display
import random, operator
#===================================================#
import numpy as np # importing numpy for matrix operations 
from scipy import *
from scipy.optimize import minimize
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython import display 
import itertools
from IPython.display import display
import random, operator
#===================================================#
seed(0) #Fix seed 

In [92]:
class Firm:
    #class attributes intialisation, to be updated with the addition of each instance
    _registry = []
    
    def __init__(self, index, capacity, max_number_of_vessel, design_speed, 
                 min_speed, max_speed, main_engine_power, aux_engine_power,
                 fix_cost, fuel_type):
        '''__init__ a method to describe the poperty that all shipping firm have'''
        self._registry.append(self)
        self.index = index
        self.capacity = capacity # Vessel capacity in TEU per firm, 
        self.max_number_of_vessel = max_number_of_vessel # # of vessels per firms
        self.design_speed = design_speed  #design speed of the vessel in knots
        self.min_speed = min_speed #min vessel speed
        self.max_speed = max_speed #max vessel speed         
        self.main_engine_power =  main_engine_power  # PS_m : main engine power in kW
        self.aux_engine_power =  aux_engine_power  # auxiliary engine power [kW] 
        self.fix_cost = fix_cost #Daily cost of vessel (USD/Day) $25,000
        self.fuel_type = fuel_type
        #self.bau_emission = bau_emission
         
   
    # Firm methods 
    def get_market_share(self , market_instance):
        '''return market share of the firm accoding to it s capacity'''
        self.transport_capacity = np.multiply(self.max_number_of_vessel , self.capacity)
        self.market_share= np.divide(self.transport_capacity, market_instance.market_capacity)
        return self.market_share
        
    def get_firm_param(self, market_instance):
        self.psy = np.multiply (self.market_share , market_instance.market_psy)
        return self.psy
    
    def get_firm_demand(self, market_instance):
        self.firm_demand = np.multiply (self.market_share , market_instance.market_demand)
        return self.firm_demand 
    
    
    def get_min_number_of_vessel(self, market_instance, operational_speed):
        self.time_at_sea = np.true_divide(market_instance.distance,  operational_speed)
        self.voyage_time = self.time_at_sea  + market_instance.port_time 
        self.number_of_trips_to_meet_demand = np.true_divide(self.firm_demand[market_instance.period_index], self.capacity)
        self.max_trips_per_vessel = np.true_divide(market_instance.annual_working_time, self.voyage_time)
        self.min_number_of_vessel_to_meet_demand = np.true_divide(self.number_of_trips_to_meet_demand, self.max_trips_per_vessel)
        return self.min_number_of_vessel_to_meet_demand, self.number_of_trips_to_meet_demand, self.time_at_sea 
    
    def get_number_of_vessel(self):
        self.number_of_vessel = np.ceil(self.min_number_of_vessel_to_meet_demand)
        return self.number_of_vessel
        
        
    def get_ship_energy_efficiency(self, market_instance):
        self.main_fuel_parameter = market_instance.SFOC_main * market_instance.eng_load_main * self.main_engine_power * 10**(-6)
        self.ship_energy = np.multiply(self.main_fuel_parameter, np.power(self.design_speed, -3) )      
        return self.ship_energy
        
    def get_main_fuel_cons(self, market_instance, operational_speed):
        self.main_fuel_cons = market_instance.distance * self.ship_energy * operational_speed**2 *self.number_of_trips_to_meet_demand
        return self.main_fuel_cons
        
    def get_aux_fuel_cons(self, market_instance): 
        self.aux_fuel_parameter = market_instance.SFOC_aux * market_instance.eng_load_aux * self.aux_engine_power * 10**(-6)
        self.aux_fuel_cons =  self.aux_fuel_parameter * self.time_at_sea 
        return self.aux_fuel_cons 
        
    def get_fuel_cost(self, market_instance):
        self.aux_fuel_cost = self.aux_fuel_cons  * market_instance.fuel_data['MGO']['price'] 
        self.main_fuel_cost = self.main_fuel_cons * market_instance.fuel_data[self.fuel_type]['price']  
        self.fuel_cost = self.main_fuel_cost + self.aux_fuel_cost
        return self.fuel_cost
        
    
    def get_total_cost(self):
        self.operating_cost = self.fix_cost * self.number_of_vessel #Fixed Cost
        self.total_cost = self.operating_cost + self.fuel_cost
        return self.total_cost 
    
    def get_revenue(self,market_instance):
        self.revenue = self.firm_demand[market_instance.period_index] * market_instance.freight_rate
        return self.revenue
    
    def get_period_profits(self):
        self.period_profit = self.revenue - self.total_cost 
        return self.period_profit
    
    def get_discounted_period_profit(self,market_instance):
        self.discounted_period_profit = (np.power(1+ market_instance.discount_rate, - market_instance.period_index),self.period_profit  )
        return self.discounted_period_profit
    
    def get_firm_carbon_emission(self, market_instance):
        self.carbon_aux_emision_factor = market_instance.fuel_data['MGO']['carbon_factor']
        self_carbon_aux_emissions = self.carbon_aux_emision_factor * self.aux_fuel_cons 

        self.carbon_main_emision_factor =  market_instance.fuel_data[self.fuel_type]['carbon_factor']
        self_carbon_main_emissions =  self.carbon_main_emision_factor * self.main_fuel_cons

        self_carbon_emissions = self_carbon_main_emissions  + self_carbon_aux_emissions
        return self_carbon_emissions
    
    def get_firm_sulfur_emission(self, market_instance):
        self.sulfur_aux_emision_factor = market_instance.fuel_data['MGO']['sulfur_factor']
        self_sulfur_aux_emissions = self.sulfur_aux_emision_factor * self.aux_fuel_cons 

        self.sulfur_main_emision_factor =  market_instance.fuel_data[self.fuel_type]['sulfur_factor']
        self_sulfur_main_emissions =  self.sulfur_main_emision_factor * self.main_fuel_cons

        self_sulfur_emissions = self_sulfur_main_emissions  + self_sulfur_aux_emissions
        return self_sulfur_emissions
    
class MarketConfig: 
    """Market Configuration and simulation game set up
    .. version: : 0.1
    
    Parameters
    ----------
    
    Attributes
    ----------
    
    """
    #class attributes
    t0 = 2016 #The starting year for evaluation of the pay-offs
    T = 35  # Planning Horizon t = 2016,...,2040 # 100 year scope 
    year = np.arange(2016, 2051)
    distance = 11810 #nautical miles #Notteboom (2006)  
    port_time = 264 #hours ==> 11 days in a year  #Notteboom (2006)
    demand_16_20 = np.array([1303780  for j in range(5)]) #in TEU #placehplder to be calibrated with the chosen route 
    annual_working_time = 6480 #hours per year, assumption   #hours per year, assumption 
    initial_freight_rate = 1800 ## in US$/TEU 
    SFOC_main = 206 #g/kWh, specific daily main engine fuel oil consumption rate
    SFOC_aux = 221 # specific fuel oil consumption of the auxiliary engine [g/kW h], 
    eng_load_main = 0.8 # % 
    eng_load_aux = 0.5 # engine load of the auxiliary engine [\%]


    def __init__(self, number_of_firms, demand_income_elasticity, demand_price_elasticity, 
                 freight_rate, fuel_data, discount_rate ):
        '''__init__ a method to describe the poperty that the shipping market and simulation game has'''
        self.number_of_firms = number_of_firms
        self.demand_income_elasticity = demand_income_elasticity #Constant income elasticity #IMF
        self.demand_price_elasticity =  demand_price_elasticity #Constant own price elasticity#IMF
        self.freight_rate = freight_rate ## in US$/TEU
        self.fuel_data = fuel_data 
        self.discount_rate = discount_rate 
        #self.bau_industry_emission =  bau_industry_emission 
        #self.bau_pollution_stock = bau_pollution_stock
        self.market_capacity = 0 
        self.pollution_stock = 0
        self.industry_damage = 0
        self.global_abatement_benefits = 0
        
    
    def get_market_capacity (self, firm_instance):
        self.market_capacity += firm_instance.max_number_of_vessel * firm_instance.capacity
        return self.market_capacity 
    
    def get_freight_rate_ratio(self): 
        self.beta = np.divide(self.freight_rate, MarketConfig.initial_freight_rate) 
        return self.beta
   
    def get_market_demand(self):
        '''compute market level demand'''
        #---------> 1.Import real GDP growth data & compute GDP ratio : Source IMF@2020 #
        G_df = pd.read_csv('./data/real_growth_rate.csv') #import IMF data
        #-------> 2.Construct GDP projection path 2016-2050 based on projection growth data 
        gdp_growth= G_df.values[:,1:] 
        g =1 +(gdp_growth/100)  
        self.gdp = np.array([100.00 for j in range(MarketConfig.T)])
        for foo in range (1,MarketConfig.T):
            self.gdp[foo] = g[:,foo]* self.gdp[foo-1]
        #print(self.gdp)
        #--------->  "compute GDP ratio based on IMF@2020"
        gdp_ratio = np.array([1.00 for j in range(MarketConfig.T)])
        for moo in range (1,MarketConfig.T):
            sub_g = g[:,0:moo+1]
            #print(sub_g)
            gdp_ratio[moo] = np.prod(sub_g)
        
        #--------->  "compute freight rate ratio"
        self.freight_rate_ratio_multiplied = np.power(self.beta, self.demand_price_elasticity)
        #print(freight_rate_ratio_multiplied)
        
        #---------> "Project Transport Demand (industry demand)"
        loo =np.multiply(np.power(gdp_ratio, self.demand_income_elasticity), self.freight_rate_ratio_multiplied ) #Will need to be updated in case fuel prices are varied over time  
        self.market_demand = np.multiply(loo, MarketConfig.demand_16_20[0])
        #print(Y) # size = 1 dimesion array with size T=35 years
        self.market_psy = np.multiply(np.power(gdp_ratio, self.demand_income_elasticity), MarketConfig.demand_16_20[0] )
        return self.market_demand, self.gdp, self.market_psy
    
    def set_period_index (self, period):
        self.period_index = period
        return self.period_index

        
    def get_industry_emission(self):
        
        pass
    
    def get_pollution_stock(self):
        pass
    
    def get_damage(self):
        pass
    
    def get_global_abatement_benefits(self):
        pass 
    
    def get_firm_abatement_benefits():
        pass

# 1. Define Chromosome 

In [113]:
Chromo = List[float] #list of 1 and 0 ?? ; need to update !! #binary encoding,..., has vessel speed and number of vessels ??
Population = List[Chromo] # list of candidate solutions generation=pop={solu1, ..., soln}={Chromosome1, ..., Chromosome}

In [114]:
###########################################################################
#===================> 0.Chromosome and GA parameter intialisation #==============================>
##########################################################################
#1. Initilize Chromosome, ie, candidate solutions placeholder list and specify type
#'''A function to generate a chromosome for one solution of foalt number '''
#random sampeling between V_min, V_max 
#which methofs to use ; uniform, ...==> Must investigate 

def generate_chromo(lower_bound:float , upper_bound:float , length: int) -> Chromo:
    return np.random.uniform(lower_bound, upper_bound, length)

# 2. Define Population  

In [115]:
###########################################################################
#========================> 2.Generating Populations #==============================>
###########################################################################
#population size at least 50 
#function to generate a population by calling generate genome multiple times  until our population has the desired size'''
def generate_population(size: int, chromo_length: int, chromo_lower_bound: float , chromo_upper_bound:float) -> Population: 
    return [generate_chromo(chromo_lower_bound, chromo_upper_bound, chromo_length) for _ in range(size)]

# 1. Fitness function 

In [35]:
###########################################################################
#==============================> 5.Population Fitness function #==============================>
###########################################################################
def fitness (chromo: Chromo, firm: Firm, sim_game: MarketConfig, max_number_of_vessel: int) -> float:
    min_number_vessels_sim , trips_to_meet_demand_sim, time_at_sea_sim  = firm.get_min_number_of_vessel(sim_game, chromo)
    number_vessels_sim  = firm.get_number_of_vessel()
    #Insert Model Constraints Here 
    if number_vessels_sim > max_number_of_vessel:
        return 0
    else:
        main_fuel_con_sim = firm.get_main_fuel_cons(sim_game, chromo)
        aux_fuel_con_sim = firm.get_aux_fuel_cons(sim_game)
        fuel_cost_sim = firm.get_fuel_cost(sim_game)
        total_cost_sim = firm.get_total_cost()
        revenues_sim = firm.get_revenue(sim_game)
        period_profits_sim = firm.get_period_profits()
        discounted_period_profits_sim = firm.get_discounted_period_profit(sim_game)
        #We will see about the carbon and sulfir emississons 
        carbon_emissions_sim = firm.get_firm_carbon_emission(sim_game)
        sulfur_emissions_sim = firm.get_firm_sulfur_emission(sim_game)
        return  discounted_period_profits_sim

In [112]:
###############################################Main########################################
#================================================================


PopulateFunc = Callable[[], Population]

def run_evolution( populate_func: PopulateFunc) -> Population:
    #1. Get the first ever population function 
    population = populate_func()
    return population 


population = run_evolution( populate_func = partial(generate_population,
                                                    size=50,
                                                    chromo_length= 1, 
                                                    chromo_lower_bound = 20 ,
                                                    chromo_upper_bound = 30
                                                   )
                          ) 

population

[array([26.35486695]),
 array([21.16654571]),
 array([20.6409789]),
 array([29.60755123]),
 array([29.62271267]),
 array([23.74742337]),
 array([23.74831752]),
 array([29.57153241]),
 array([26.06364861]),
 array([24.23621269]),
 array([27.82315469]),
 array([28.17197947]),
 array([27.33707888]),
 array([24.75995331]),
 array([22.07549265]),
 array([24.51617115]),
 array([20.91681338]),
 array([24.01293317]),
 array([23.67767768]),
 array([27.76811049]),
 array([20.96521834]),
 array([27.29015113]),
 array([22.73595533]),
 array([29.06789824]),
 array([28.95410004]),
 array([23.89917267]),
 array([26.44475606]),
 array([22.3279895]),
 array([24.29550598]),
 array([29.04292824]),
 array([27.52848739]),
 array([27.10242939]),
 array([29.22345594]),
 array([21.25191705]),
 array([23.72801494]),
 array([23.69302549]),
 array([23.35567862]),
 array([22.21218229]),
 array([29.55932543]),
 array([27.79565751]),
 array([25.20168719]),
 array([27.64074708]),
 array([27.25692502]),
 array([26.81

In [ ]:
PopulateFunc = Callable[[], Population] # a population fn that takes nothing and returns new solutiobns #=Callable[[Input], Output]

FitnessFunc = Callable[[Chromo], int] # a fitness function that takes a genome and returns a fitness value to make the correct choice


In [36]:
#Survival of the fittest  and parent selection 
#tournement selction with constraint  : randomaly select pairs and pick the fittest 
#@ the end of the selection, we result in pairs : parents:
#1) prefer a feasible solution 
#2) if both feasible, chosse better objective
#3) if both infeasibe, chose the one with the less constraint variation
#The fitness of the chromose is the chromosome's weight in the selection process
# we draw 2 to get a pair 
#choices fn from python random module allow us to assign the fitness as weights for each element it can choose from
#fitness of a genome = the genome's weights
# k=2, we draw 2 from our population
#Add weights to each element it can choose from    
# Weight Distribution
###########################################################################
#========================> 6.Select function #==============================>
###########################################################################
def selection_pair(population: Population, fitness_func: FitnessFunc) -> Population:
    '''select a pair of chromo which will be the parent of 2 new chromo for the next generation, Solution with higher fitness should be more likely to be chosen for reproduction'''
    return choices(
        population = population,
        weights=[fitness_func(chromo) for chromo in population],
        k=2
        )

In [ ]:
###########################################################################
#======================>3.Single point Crossover function #=========================>
###########################################################################
def single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:    
    # lengths of genome is at least 2 otherwise there would be no point to cut the genome in half
    length = len(a)
    if length < 2:
        return a, b

    p = randint(1, length - 1) #randomly choose an index to cut the genomes in half it 
    return a[0:p] + b[p:], b[0:p] + a[p:] # the first new solution =  the first 1/2 of the 1st genome a and the rest of the second genome b
#2nd new solution: 1st part of genome b and 2nd part of genome a
    

    #==================================================================#     
#Given parents  a : chromo1 and b:chromo2 :
#randomly select a child between
# min(a,b) - alpha *[max(a,b) - min (a,b)] &  max(a,b) + alpha *[max(a,b) - min (a,b)]
# best practice alpha = 0.5    
    
def blend_crossover (a: Chromo, b: Chromo, 0.5: float) -> Tuple[Chromo, Chromo]:
    '''generate a new solution for the next generation, takes 2 chromo as parameters and returns 2 chromo as output'''
     return np.random.uniform(min(a,b) -  0.5 * (max(a,b) - min (a,b)), max(a,b) +  0.5 * (max(a,b) - min (a,b)), 2) 
    
#==================================================================#
def linear_crossover (a: Chromo, b: Chromo) -> Tuple[Chromo, Chromo]:
    c =  0.5 * a + 0.5 * b
    d = 1.5 * a - 0.5 * b
    e = -0.5 * a - 1.5 * b
    children = [a,b,c,d,e]
    return  max(fitness_func(crossed) for crossed in children)

#Given parents  a : chromo1 and b:chromo2 :
#create 3 solutions: 
# 0.5 a + 0.5 b
# 1.5 a - 0.5 b
#-0.5 a - 1.5 b
# ====> Children = the 2 best solutions among the 5 



In [ ]:
#mutation: randomly perturb the values
# Uniform variation: x_new = xi + (ri - 0.5) delta_i
#Normal distribution : A gaussian with mean 0 and and a chosen standard deviation   x_new = xi +N(0, sigma) 
#1. Random mutation 
#2. Non-Uniform Mutation
#3.Normally distributed mutation 
###########################################################################
#==============================> 4.Mutation function #==============================>
###########################################################################
def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
    '''takes a genome and w/ a certain proba changes 1 to 0 and 0 to 1 @ random positions'''
    ''''''
    for _ in range(num):
        index = randrange(len(genome)) # generate a random index
        genome[index] = genome[index] if random() > probability else abs(genome[index] - 1) #if random returns a value>> prob, we leave it alone otherwise it 
        #falls within the mutation proba and we need to change it into the | current value - 1|
    return genome 

### my version 
def mutation(chromo: Chromo, num: int = 1, probability: float = 0.5) -> Chromo:
        chromo = chromo if random() > probability else 
        blaa 
        
        #if random returns a value>> prob, we leave it alone otherwise it 
        #falls within the mutation proba and we need to change it into the | current value - 1|
    return chromo 

In [ ]:
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Chromo, Chromo]]  #takes a population and a fitness fn to select 2 solutions to be the parents of our next generaation solution

CrossoverFunc = Callable[[Chromo, Chromo], Tuple[Chromo, Chromo]] #takes 2 genomes and returns 2 new genomes 

MutationFunc = Callable[[Chromo], Chromo] #takes 1 genome and sometimes returns a modified one

PrinterFunc = Callable[[Population, int, FitnessFunc], None]


In [ ]:
###########################################################################
#==============================> Run algorithm #==============================>
###########################################################################
def run_evolution(
        populate_func: PopulateFunc,
        fitness_func: FitnessFunc,
        fitness_limit: int,
        selection_func: SelectionFunc = selection_pair,
        crossover_func: CrossoverFunc = single_point_crossover,
        mutation_func: MutationFunc = mutation,
        generation_limit: int = 100,
        printer: Optional[PrinterFunc] = None) \
        -> Tuple[Population, int]:
    
    #1. Get the first ever population function 
    population = populate_func()

    # 2. loop for generation limit time 
    for i in range(generation_limit):
        #1. Sort population 
        population = sorted(
            population, 
            key=lambda chromo: fitness_func(chromo), reverse=True) ##Sort population by fitness so that top solutions are in the first indices of our list of genomes 


        if printer is not None:
            printer(population, i, fitness_func)

        if fitness_func(population[0]) >= fitness_limit:
            break  #check if we have already reached the fitness limit and return early from our loop or if we want to implement elitism  

        next_generation = population[0:2] #Keep the best 2 chromo for the next generation,#or Implement elitism, ie keep our top 2 solutions for our next generation
        
        #Generate all other new chromo for our next generation 
        #Loop for generation limot times (100 generation - 1 )
        # generate all other new chromo for our next generations 
        #pick 2 parents and get 2 new chromo everytime
        # loop for 1/2 the length of a genration to get as many chromo in our next generation as
        #as before but b/c we already copied the 2 top chromo fron our last generation we can 
        #save one more loop 
        for j in range(int(len(population) / 2) - 1):
            parents = selection_func(population, fitness_func) #in each loop, pick 2 parents 
            offspring_a, offspring_b = crossover_func(parents[0], parents[1]) #get 2 child chromo for our next generation
            offspring_a = mutation_func(offspring_a) # mutate each offspring chromo to expand the variety of chromos we get 
            offspring_b = mutation_func(offspring_b)
            next_generation += [offspring_a, offspring_b] #next generation

        population = next_generation #update current population with our next generation and start into the next round of the algo by sorting the population and checking if we reached our fitness limit 

    
    #Sort the population one last time in case we run out of generation  
    population = sorted(
            population, 
            key=lambda chromo: fitness_func(chromo), reverse=True)

    return population, i #the i is to distinguish wether the algorithm exausted the generation limit or actually found the solultion that meets our fitness criteria 

In [ ]:
population, generations = run_evolution(
    populate_func = partial(generate_population, 
                            size=50,
                            chromo_length= 1),
    
    fitness_func = partial(fitness,
                           Firm = firm_1,
                           MarketConfig = sim_game, 
                           max_number_of_vessel = max_number_of_vessel),
    fitness_limit = inf,
    generation_limit = 100) 

In [ ]:
###########################################################################
#==============================> 6.Printing #==============================>
###########################################################################
def chromo_to_string(chromo: Chromo) -> str:
    return "".join(map(str, chromo))


def print_stats(population: Population, generation_id: int, fitness_func: FitnessFunc):
    print("GENERATION %02d" % generation_id)
    print("=============")
    print("Population: [%s]" % ", ".join([chromo_to_string(chromo) for chromo in population]))
    print("Avg. Fitness: %f" % (population_fitness(population, fitness_func) / len(population)))
    sorted_population = sort_population(population, fitness_func)
    print(
        "Best: %s (%f)" % (chromo_to_string(sorted_population[0]), fitness_func(sorted_population[0])))
    print("Worst: %s (%f)" % (chromo_to_string(sorted_population[-1]),
                              fitness_func(sorted_population[-1])))
    print("")

    return sorted_population[0]

In [ ]:
###########################################################################
#==============================> 5.Population Fitness function #==============================>
###########################################################################
def population_fitness(population: Population, fitness_func: FitnessFunc) -> int:
    '''the fitness fn determm how good a candidate given solution (genome) is'''
    '''insert the subroutine constraint'''
    return sum([fitness_func(genome) for genome in population])

In [ ]:
populate_func: PopulateFunc

fitness_func: FitnessFunc 
selection_func: SelectionFunc = selection_pair
crossover_func: CrossoverFunc = single_point_crossover
mutation_func: MutationFunc = mutation
printer: Optional[PrinterFunc] = None)

In [ ]:
class Optimisation:
    def __init__(self, length):
        self.chromo =
        self.population = 
        pass
    def __str__(self):
        

In [19]:

##########################################################################
#========================> 2.Initial Population#====================
###########################################################################
# #random samepling 
# #Maybe use latin hyperkeep sampeling approach 
# print(__doc__)
# import numpy as np
# np.random.seed(123)
# import matplotlib.pyplot as plt
# from skopt.space import Space
# from skopt.sampler import Sobol
# from skopt.sampler import Lhs
# from skopt.sampler import Halton
# from skopt.sampler import Hammersly
# from skopt.sampler import Grid
# from scipy.spatial.distance import pdist
# lhs = Lhs(criterion="maximin", iterations=10000)
# x = lhs.generate(space.dimensions, n_samples)
# plot_searchspace(x, 'maximin LHS')
# pdist_data.append(pdist(x).flatten())
# x_label.append("maximin")
